In [1]:
import os

os.environ["OMP_NUM_THREADS"] = "8"
os.environ["OPENBLAS_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
os.environ["VECLIB_MAXIMUM_THREADS"] = "8"
os.environ["NUMEXPR_NUM_THREADS"] = "8"
import numpy as np
import cobra
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
from helper_function import *
plt.rcParams['font.family'] = 'Arial'

In [ ]:
model = cobra.io.load_matlab_model('universal_model.mat')

# with open('Ecoli_min_M.pkl', 'rb') as pickle_file:
#     data = pickle.load(pickle_file)
#     all_total_nutrient_count = data['all_total_nutrient_count']
#     all_total_intake = data['all_total_intake']
#     all_growth_rate = data['all_growth_rate']
#     all_met_flux = data['all_met_flux']
#     del data


with open("FBAsolution/M9_sufficient_423_1_proteome_weakbg.pkl", "rb") as f:
    data = pickle.load(f)
    all_met_flux = data["all_met_flux"]
    all_growth_rate = data["all_growth_rate"]
    all_medium = data["all_random_medium"]

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-29


In [3]:
S_df = cobra.util.create_stoichiometric_matrix(model, array_type='DataFrame')
reaction_list = list(S_df.columns)
metabolite_list = list(S_df.index)

In [4]:
S = S_df.to_numpy()

In [ ]:
currency_mets = [
    'ctp[c]', 'cdp[c]', 'pi[c]', 'h[c]', 'gtp[c]', 'gdp[c]', 'itp[c]', 'idp[c]', 'nadph[c]', 'nadp[c]', 'fadh2[c]', 'fad[c]',
    'fmnh2[c]', 'fmn[c]', 'mql8[c]', 'mqn8[c]', '2dmmql8[c]', '2dmmq8[c]', 'accoa[c]', 'coa[c]', 'nad[c]', 'nadh[c]', 'atp[c]', 'adp[c]', 'ppi[c]','pppi[c]'
    'q8[c]', 'q8h2[c]','cbi[c]', 'amp[c]', 'udp[c]','cmp[c]', 'o2[c]','h2o[c]','co2[c]',

    'dmbzid[c]','PGPm1[c]', 'apoACP[c]', 'ACP[c]','malACP[c]','dnarep[c]','proteinsynth[c]','rnatrans[c]',

    'ctp[p]', 'cdp[p]', 'pi[p]', 'h[p]', 'gtp[p]', 'gdp[p]', 'itp[p]', 'idp[p]', 'nadph[p]', 'nadp[p]', 'fadh2[p]', 'fad[p]',
    'fmnh2[p]', 'fmn[p]', 'mql8[p]', 'mqn8[p]', '2dmmql8[p]', '2dmmq8[p]', 'accoa[p]', 'coa[p]', 'nad[p]', 'nadh[p]', 'atp[p]', 'adp[p]', 'ppi[p]','pppi[p]'
    'q8[p]', 'q8h2[p]','cbi[p]', 'amp[p]', 'udp[p]','cmp[p]', 'o2[p]','h2o[p]','co2[p]',

    'dmbzid[p]','PGPm1[p]', 'apoACP[p]', 'ACP[p]','malACP[p]','dnarep[p]','proteinsynth[p]','rnatrans[p]',

    'ctp[e]', 'cdp[e]', 'pi[e]', 'h[e]', 'gtp[e]', 'gdp[e]', 'itp[e]', 'idp[e]', 'nadph[e]', 'nadp[e]', 'fadh2[e]', 'fad[e]',
    'fmnh2[e]', 'fmn[e]', 'mql8[e]', 'mqn8[e]', '2dmmql8[e]', '2dmmq8[e]', 'accoa[e]', 'coa[e]', 'nad[e]', 'nadh[e]', 'atp[e]', 'adp[e]', 'ppi[e]','pppi[e]',
    'q8[e]', 'q8h2[e]','cbi[e]', 'amp[e]', 'udp[e]','cmp[e]', 'o2[e]','h2o[e]','co2[e]',

    'dmbzid[e]','PGPm1[e]', 'apoACP[e]', 'ACP[e]','malACP[e]','dnarep[e]','proteinsynth[e]','rnatrans[e]',
    'h2o[p]','co2[p]','h[p]','pi[p]','co2[p]',
    # 'nh4[c]',
    # 'mn2[p]', 'cu2[p]', 'pi[p]','cobalt2[p]',  'mg2[p]','co2[p]', 'o2[p]', 'cl[p]','zn2[p]', 'so4[p]', 'fe3[p]',
    #  'k[p]', 'nh4[p]','ca2[p]','fe2[p]',
    'h2o[e]','co2[e]','biomass[e]'
    
    ]
for met in currency_mets:
    if met in metabolite_list:
        S[metabolite_list.index(met), :] = 0

In [ ]:
import pandas as pd
modelname = pd.read_excel('ModelList.xlsx')
modelname = np.array(modelname['ModelName'])

In [ ]:
import gc
from scipy import sparse

comparison = []
for j, met_flux in enumerate(all_met_flux[0]):
    print(j)
    # if os.path.exists('flow_graph_uni_423/'+modelname[j][1:-5]+'.pkl'):
    #     print(f"already exists. Skipping...")
    #     continue

    try:

        M1 = construct_mass_flow_graph(S, met_flux, reaction_list)
        G1 = construct_networkx_graph(M1, reaction_list)

        sparse_M1 = sparse.csr_matrix(M1)


        sparse.save_npz('flow_graph_423/'+ modelname[j][1:-5] +'.npz', sparse_M1)

        with open('flow_graph_423/'+ modelname[j][1:-5] +'.pkl', 'wb') as f:
            pickle.dump(G1, f)



        del M1, G1
        # 强制垃圾回收
        gc.collect()

    except Exception as e:
        print(f"Error on {j}: {e}")
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


KeyboardInterrupt: 